In [1]:
# Both Data Integration:

In [2]:
# Configuration 
import configparser

# Create ConfigParser object
config = configparser.ConfigParser()
config.read("config.ini", encoding="utf-8")  # Ensure UTF-8 encoding

df_1_pest_pkl = config.get("1_PEST", "df_1_pest_pkl")
df_2_weather_pkl = config.get("2_WEATHER", "df_2_weather_pkl")

weather_agg_window_days = config.getint("3_BOTH", "weather_agg_window_days")
selected_pest_columns = config.get("3_BOTH", "selected_pest_columns").replace("\n", "").split(", ")
selected_weather_columns = config.get("3_BOTH", "selected_weather_columns").replace("\n", "").split(", ")
drop_weather_columns = config.get("3_BOTH", "drop_weather_columns").replace("\n", "").split(", ")
sort_by = config.get("3_BOTH", "sort_by").replace("\n", "").split(", ")
rolling_avg_cols = config.get("3_BOTH", "rolling_avg_cols").replace("\n", "").split(", ")
rolling_sum_cols = config.get("3_BOTH", "rolling_sum_cols").replace("\n", "").split(", ")
group_by_columns_rolling = config.get("3_BOTH", "group_by_columns_rolling").replace("\n", "").split(", ")
join_type = config.get("3_BOTH", "join_type")
join_on_pest = config.get("3_BOTH", "join_on_pest").replace("\n", "").split(", ")
join_on_weather = config.get("3_BOTH", "join_on_weather").replace("\n", "").split(", ")

df_3_both_pkl = config.get("3_BOTH", "df_3_both_pkl")
df_3_both_csv = config.get("3_BOTH", "df_3_both_csv")

In [3]:
print(weather_agg_window_days)

4


In [4]:
import pandas as pd
import time

# unpickle - pandas dataframe
df_pest = pd.read_pickle(df_1_pest_pkl)
df_weather = pd.read_pickle(df_2_weather_pkl)

print(df_pest.shape)
print(df_weather.shape)

(530, 20)
(42260, 34)


In [5]:
df_pest

,delomrade,lan,latitud,longitud,groda,sort,jordart,sadatum,skordear,graderingsdatum,graderingstyp,forforfrukt,forfrukt,ekologisk,plojt,utvecklingsstadium,Bladfläcksvampar,Gulrost,Svartpricksjuka,Sädesbladlus
0,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-01,Veckovis,Vårkorn,Vårkorn,False,True,26,0.0,0.0,0.0,0.0
1,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-08,Veckovis,Vårkorn,Vårkorn,False,True,30,0.0,0.0,0.0,0.0
2,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-15,Veckovis,Vårkorn,Vårkorn,False,True,32,0.0,0.0,0.0,0.0
3,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-22,Veckovis,Vårkorn,Vårkorn,False,True,33,0.0,0.0,0.0,0.0
4,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-29,Veckovis,Vårkorn,Vårkorn,False,True,37,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,None,2023-09-22,2024,2024-04-15,Veckovis,None,Höstraps,False,None,22,43.0,0.0,43.0,0.0
526,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,None,2023-09-22,2024,2024-04-22,Veckovis,None,Höstraps,False,None,22,40.0,0.0,40.0,0.0
527,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,None,2023-09-22,2024,2024-04-26,Veckovis,None,Höstraps,False,None,30,40.0,0.0,40.0,0.0
528,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,None,2023-09-22,2024,2024-05-03,Veckovis,None,Höstraps,False,None,31,38.0,0.0,38.0,0.0


In [6]:
df_weather

,STATIONID,POINTNAME,MUNICID,REGIONNAME,WGS84N,WGS84E,DATAFROM,DATATO,ARABLEPCT,MESANGRIDID,...,WDCM,Q0,BT,G0,G3,G5,G7,G8,G10,WSTNID
0,103357.0,Botkyrka kommun,127.0,Stockholms län,59.140,17.906,20091001.0,20250327.0,10.9,103357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,250317.0,Botkyrka kommun,127.0,Stockholms län,59.100,17.883,20180501.0,20250327.0,40.5,250317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,250945.0,Botkyrka kommun,127.0,Stockholms län,59.127,17.787,20180501.0,20250327.0,49.9,250945.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,252205.0,Botkyrka kommun,127.0,Stockholms län,59.177,17.791,20180501.0,20250327.0,67.0,252205.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,252835.0,Botkyrka kommun,127.0,Stockholms län,59.202,17.793,20180501.0,20250327.0,38.4,252835.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42255,NaN,NaN,NaN,NaN,69.042,20.262,NaN,NaN,NaN,NaN,...,224.0,0.0,100.0,1469.0,1031.0,776.0,554.0,458.0,294.0,110070.0
42256,NaN,NaN,NaN,NaN,69.042,20.262,NaN,NaN,NaN,NaN,...,215.0,0.0,0.0,1469.0,1031.0,776.0,554.0,458.0,294.0,110070.0
42257,NaN,NaN,NaN,NaN,69.042,20.262,NaN,NaN,NaN,NaN,...,199.0,0.0,0.0,1469.0,1031.0,776.0,554.0,458.0,294.0,110070.0
42258,NaN,NaN,NaN,NaN,69.042,20.262,NaN,NaN,NaN,NaN,...,222.0,0.0,100.0,1469.0,1031.0,776.0,554.0,458.0,294.0,110070.0


In [7]:
df_weather['DAY'].unique()

array([nan, '2016-01-01', '2016-01-02', ..., '2024-12-29', '2024-12-30',
       '2024-12-31'], dtype=object)

In [8]:
df_pest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   delomrade           530 non-null    object 
 1   lan                 530 non-null    object 
 2   latitud             530 non-null    object 
 3   longitud            530 non-null    object 
 4   groda               530 non-null    object 
 5   sort                530 non-null    object 
 6   jordart             162 non-null    object 
 7   sadatum             264 non-null    object 
 8   skordear            530 non-null    int64  
 9   graderingsdatum     530 non-null    object 
 10  graderingstyp       530 non-null    object 
 11  forforfrukt         152 non-null    object 
 12  forfrukt            354 non-null    object 
 13  ekologisk           530 non-null    bool   
 14  plojt               161 non-null    object 
 15  utvecklingsstadium  530 non-null    int64  
 16  Bladfläc

In [9]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42260 entries, 0 to 42259
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   STATIONID    12668 non-null  float64
 1   POINTNAME    12668 non-null  object 
 2   MUNICID      12668 non-null  float64
 3   REGIONNAME   12668 non-null  object 
 4   WGS84N       42260 non-null  float64
 5   WGS84E       42260 non-null  float64
 6   DATAFROM     12668 non-null  float64
 7   DATATO       12668 non-null  float64
 8   ARABLEPCT    12668 non-null  float64
 9   MESANGRIDID  12668 non-null  float64
 10  latitud      42260 non-null  object 
 11  longitud     42260 non-null  object 
 12  DAY          29592 non-null  object 
 13  HOUR         29592 non-null  float64
 14  TM           29592 non-null  float64
 15  TX           29592 non-null  float64
 16  TN           29592 non-null  float64
 17  RR           29592 non-null  float64
 18  RRA          29592 non-null  float64
 19  UM  

In [10]:
# Convert 'graderingsdatum' in df_pest to datetime
df_pest['graderingsdatum'] = pd.to_datetime(df_pest['graderingsdatum'], errors='coerce')

# Convert 'DAY' in df_weather to datetime
df_weather['DAY'] = pd.to_datetime(df_weather['DAY'], errors='coerce')

# Check if the conversion was successful
print(df_pest['graderingsdatum'].head())
print(df_weather['DAY'].head())

0   2017-05-01
1   2017-05-08
2   2017-05-15
3   2017-05-22
4   2017-05-29
Name: graderingsdatum, dtype: datetime64[ns]
0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: DAY, dtype: datetime64[ns]


In [11]:
# Select specific columns:

df_pest = df_pest[selected_pest_columns]
df_weather = df_weather[selected_weather_columns]

print(selected_pest_columns)
print(selected_weather_columns)

['delomrade', 'lan', 'latitud', 'longitud', 'groda', 'sort', 'jordart', 'skordear', 'graderingsdatum', 'graderingstyp', 'utvecklingsstadium', 'Bladfläcksvampar', 'Gulrost', 'Svartpricksjuka', 'Sädesbladlus']
['DAY', 'WGS84N', 'WGS84E', 'latitud', 'longitud', 'TM', 'TX', 'TN', 'UM', 'UX', 'UN', 'FM2', 'CLDM', 'Q0', 'RR', 'BT', 'G0', 'G3', 'G5', 'G7', 'G8', 'G10']


In [12]:
# Weather Data Calculation by Rolling Data:

# Sort data by location and date for rolling calculation
df_weather = df_weather.sort_values(by=sort_by)

# Define columns for rolling average and sum
rolling_avg_cols = rolling_avg_cols
rolling_sum_cols = rolling_sum_cols

# Compute rolling averages
for col in rolling_avg_cols:
    df_weather[f"{col}_7d_avg"] = (
        df_weather.groupby(group_by_columns_rolling)[col]
        .transform(lambda x: x.rolling(window=weather_agg_window_days, min_periods=1).mean())
        .round(2)
    )

# Compute rolling sums
for col in rolling_sum_cols:
    df_weather[f"{col}_7d_sum"] = (
        df_weather.groupby(group_by_columns_rolling)[col]
        .transform(lambda x: x.rolling(window=weather_agg_window_days, min_periods=1).sum())
        .round(2)
    )

# Check updated dataframe
print(df_weather.head())

df_weather.shape

  DAY  WGS84N  WGS84E  latitud longitud  TM  TX  TN  UM  UX  ...  CLDM_7d_avg  \
0 NaT  59.140  17.906  2441000      000 NaN NaN NaN NaN NaN  ...          NaN   
1 NaT  59.100  17.883  2441000      000 NaN NaN NaN NaN NaN  ...          NaN   
2 NaT  59.127  17.787  2441000      000 NaN NaN NaN NaN NaN  ...          NaN   
3 NaT  59.177  17.791  2441000      000 NaN NaN NaN NaN NaN  ...          NaN   
4 NaT  59.202  17.793  2441000      000 NaN NaN NaN NaN NaN  ...          NaN   

   Q0_7d_avg  RR_7d_sum  BT_7d_sum  G0_7d_sum  G3_7d_sum  G5_7d_sum  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   
2        NaN        NaN        NaN        NaN        NaN        NaN   
3        NaN        NaN        NaN        NaN        NaN        NaN   
4        NaN        NaN        NaN        NaN        NaN        NaN   

   G7_7d_sum  G8_7d_sum  G10_7d_sum  
0        NaN        NaN         NaN  
1        N

(42260, 39)

In [13]:
df_weather.drop(drop_weather_columns, axis=1, inplace=True)

In [14]:
df_weather

,DAY,WGS84N,WGS84E,latitud,longitud,TM_7d_avg,TX_7d_avg,TN_7d_avg,UM_7d_avg,UX_7d_avg,...,CLDM_7d_avg,Q0_7d_avg,RR_7d_sum,BT_7d_sum,G0_7d_sum,G3_7d_sum,G5_7d_sum,G7_7d_sum,G8_7d_sum,G10_7d_sum
0,NaT,59.140,17.906,2441000,000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,59.100,17.883,2441000,000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaT,59.127,17.787,2441000,000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,59.177,17.791,2441000,000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaT,59.202,17.793,2441000,000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42255,2024-12-27,69.042,20.262,6140000,382000,-5.87,-3.90,-8.60,86.02,92.25,...,86.25,0.0,39.2,100.0,5876.0,4124.0,3104.0,2216.0,1832.0,1176.0
42256,2024-12-28,69.042,20.262,6140000,382000,-3.95,-1.90,-6.85,86.17,92.50,...,86.25,0.0,42.3,100.0,5876.0,4124.0,3104.0,2216.0,1832.0,1176.0
42257,2024-12-29,69.042,20.262,6140000,382000,-3.97,-2.60,-5.95,85.82,92.25,...,84.75,0.0,40.2,100.0,5876.0,4124.0,3104.0,2216.0,1832.0,1176.0
42258,2024-12-30,69.042,20.262,6140000,382000,-4.45,-2.95,-6.25,86.42,93.25,...,85.75,0.0,33.5,200.0,5876.0,4124.0,3104.0,2216.0,1832.0,1176.0


In [15]:
df_weather['DAY'].unique()

<DatetimeArray>
[                'NaT', '2016-01-01 00:00:00', '2016-01-02 00:00:00',
 '2016-01-03 00:00:00', '2016-01-04 00:00:00', '2016-01-05 00:00:00',
 '2016-01-06 00:00:00', '2016-01-07 00:00:00', '2016-01-08 00:00:00',
 '2016-01-09 00:00:00',
 ...
 '2024-12-22 00:00:00', '2024-12-23 00:00:00', '2024-12-24 00:00:00',
 '2024-12-25 00:00:00', '2024-12-26 00:00:00', '2024-12-27 00:00:00',
 '2024-12-28 00:00:00', '2024-12-29 00:00:00', '2024-12-30 00:00:00',
 '2024-12-31 00:00:00']
Length: 3289, dtype: datetime64[ns]

In [16]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42260 entries, 0 to 42259
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DAY          29592 non-null  datetime64[ns]
 1   WGS84N       42260 non-null  float64       
 2   WGS84E       42260 non-null  float64       
 3   latitud      42260 non-null  object        
 4   longitud     42260 non-null  object        
 5   TM_7d_avg    29592 non-null  float64       
 6   TX_7d_avg    29592 non-null  float64       
 7   TN_7d_avg    29592 non-null  float64       
 8   UM_7d_avg    29592 non-null  float64       
 9   UX_7d_avg    29592 non-null  float64       
 10  UN_7d_avg    29592 non-null  float64       
 11  FM2_7d_avg   29592 non-null  float64       
 12  CLDM_7d_avg  29592 non-null  float64       
 13  Q0_7d_avg    29592 non-null  float64       
 14  RR_7d_sum    29592 non-null  float64       
 15  BT_7d_sum    29592 non-null  float64       
 16  G0_7

In [17]:
# Join Pest & Weather Data:

# Perform inner join
df_both = df_pest.merge(
    df_weather,
    left_on=join_on_pest,
    right_on=join_on_weather,
    how=join_type
)

# Display the resulting DataFrame
print(df_both.shape)

(520, 35)


In [18]:
df_both = df_both.sort_values(by=['latitud', 'longitud', 'graderingsdatum'], ascending=True)

In [19]:
df_both

,delomrade,lan,latitud,longitud,groda,sort,jordart,skordear,graderingsdatum,graderingstyp,...,CLDM_7d_avg,Q0_7d_avg,RR_7d_sum,BT_7d_sum,G0_7d_sum,G3_7d_sum,G5_7d_sum,G7_7d_sum,G8_7d_sum,G10_7d_sum
0,SV Skåne,Skåne län,6135000,397000,Höstvete,Elixer,None,2017,2017-04-17,Veckovis,...,60.75,12.08,0.1,0.0,56.0,4.0,0.0,0.0,0.0,0.0
1,SV Skåne,Skåne län,6135000,397000,Höstvete,Mariboss,None,2017,2017-04-17,Veckovis,...,60.75,12.08,0.1,0.0,56.0,4.0,0.0,0.0,0.0,0.0
2,SV Skåne,Skåne län,6135000,397000,Höstvete,Praktik,None,2017,2017-04-17,Veckovis,...,60.75,12.08,0.1,0.0,56.0,4.0,0.0,0.0,0.0,0.0
3,SV Skåne,Skåne län,6135000,397000,Höstvete,Torp,None,2017,2017-04-17,Veckovis,...,60.75,12.08,0.1,0.0,56.0,4.0,0.0,0.0,0.0,0.0
4,SV Skåne,Skåne län,6135000,397000,Höstvete,Elixer,None,2017,2017-04-24,Veckovis,...,70.25,12.75,2.1,0.0,60.0,4.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,None,2024,2024-04-15,Veckovis,...,80.75,11.93,5.4,100.0,54.0,3.0,0.0,0.0,0.0,0.0
516,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,None,2024,2024-04-22,Veckovis,...,17.50,24.60,0.0,0.0,60.0,4.0,0.0,0.0,0.0,0.0
517,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,None,2024,2024-04-26,Veckovis,...,65.00,15.58,4.6,300.0,63.0,4.0,0.0,0.0,0.0,0.0
518,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,None,2024,2024-05-03,Veckovis,...,70.50,16.23,5.5,100.0,99.0,17.0,2.0,0.0,0.0,0.0


In [20]:
df_both.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   delomrade           520 non-null    object        
 1   lan                 520 non-null    object        
 2   latitud             520 non-null    object        
 3   longitud            520 non-null    object        
 4   groda               520 non-null    object        
 5   sort                520 non-null    object        
 6   jordart             152 non-null    object        
 7   skordear            520 non-null    int64         
 8   graderingsdatum     520 non-null    datetime64[ns]
 9   graderingstyp       520 non-null    object        
 10  utvecklingsstadium  520 non-null    int64         
 11  Bladfläcksvampar    518 non-null    float64       
 12  Gulrost             520 non-null    float64       
 13  Svartpricksjuka     518 non-null    float64       

In [22]:
# pickle - pandas dataframe
df_both.to_pickle(df_3_both_pkl)

# Save the DataFrame to a CSV file with UTF-8 encoding
df_both.to_csv(df_3_both_csv, index=False, encoding='utf-8-sig')  # Ensure proper encoding for special characters

print(f"Data pickled to {df_3_both_pkl} and saved to {df_3_both_csv}")

Data saved to df_3_both_t1.csv
